<a href="https://colab.research.google.com/github/rawa-elargab/Amazon-Discount-EDA/blob/main/Amazon_Discounts_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inspiration and objective
The objective is to find the best products at good price with discounts

In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import os

In [2]:
path = "/content/Amazon-Products.csv"
df = pd.read_csv(path, low_memory=False)
df.head()

,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"
3,3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990"
4,4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790"


In [3]:
# Knowing the number of rows and columns
df.shape

(37165, 10)

In [4]:
# Checking the missing values
df.isnull().sum()

Unnamed: 0           0
name                 0
main_category        1
sub_category         1
image                1
link                 1
ratings           4675
no_of_ratings     4675
discount_price    2757
actual_price       629
dtype: int64

In [5]:
# Droping the columns with 70% or more missing data
perc = 70.0 
min_count =  int(((100 - perc)/100) * df.shape[1] + 1)
mod_df = df.dropna(axis = 1, thresh = min_count)

In [6]:
# Number of columns are reduced significantly from 895 to 14
mod_df.shape

(37165, 10)

In [7]:
# Checking the column name
mod_df.columns

Index(['Unnamed: 0', 'name', 'main_category', 'sub_category', 'image', 'link',
       'ratings', 'no_of_ratings', 'discount_price', 'actual_price'],
      dtype='object')

In [8]:
# Checking the data type, missing values in remaining columns
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37165 entries, 0 to 37164
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      37165 non-null  int64 
 1   name            37165 non-null  object
 2   main_category   37164 non-null  object
 3   sub_category    37164 non-null  object
 4   image           37164 non-null  object
 5   link            37164 non-null  object
 6   ratings         32490 non-null  object
 7   no_of_ratings   32490 non-null  object
 8   discount_price  34408 non-null  object
 9   actual_price    36536 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.8+ MB


# Clean numeric columns and set data types
The columns actual_price, discount_price, no_of_ratings and ratings have wrong datatype.
The datatype given is object but we want them to be int or float. Let us correct it.<br>
The column amazon_category_and_sub_category has multiple values. In order to clean the data we will seperate them to individual columns.<br>

In [9]:
# Removing the ₹ sign
mod_df["discount_price"] = mod_df["discount_price"].str.split(" ", expand = True).get(0).str.split("₹", expand = True).get(1)
mod_df["actual_price"] = mod_df["actual_price"].str.split(" ", expand = True).get(0).str.split("₹", expand = True).get(1)

In [10]:
# Change commas to dots and change the type to float
mod_df['discount_price'] = mod_df["discount_price"].str.replace(',', '').astype(float)
mod_df["actual_price"] = mod_df["actual_price"].str.replace(',', '').astype(float)

There are values in the 'ratings' column that cannot be represented as a number. We replace them with '0.0'.


In [11]:
# Modify ratings values
mod_df['ratings'].unique()

array(['4.2', '4.0', '4.1', '4.3', '3.9', '3.8', '3.5', nan, '4.6', '3.3',
       '3.4', '3.7', '2.9', '5.0', '4.4', '3.6', '2.7', '4.5', '3.0',
       '3.1', '3.2', '4.8', '4.7', '2.5', '1.0', '2.6', '2.8', '2.3',
       '1.7', 'Get', '1.8', '2.4', '4.9', '2.2', '1.6', '1.9', '2.0',
       '1.4', '2.1', 'FREE', '1.2', '1.3', '1.5'], dtype=object)

In [12]:
# Extract the digits and change the type to float
mod_df['ratings'] = mod_df['ratings'].replace(['Get','FREE','₹68.99', '₹65','₹70', '₹100', '₹99', '₹2.99'], '0.0')
mod_df['ratings'] = mod_df["ratings"].astype(float)
mod_df['ratings'].unique()

array([4.2, 4. , 4.1, 4.3, 3.9, 3.8, 3.5, nan, 4.6, 3.3, 3.4, 3.7, 2.9,
       5. , 4.4, 3.6, 2.7, 4.5, 3. , 3.1, 3.2, 4.8, 4.7, 2.5, 1. , 2.6,
       2.8, 2.3, 1.7, 0. , 1.8, 2.4, 4.9, 2.2, 1.6, 1.9, 2. , 1.4, 2.1,
       1.2, 1.3, 1.5])

#  Preprocess Rating column
The 'no_of_ratings' column is converted to the float type in two steps: first, a new boolean column is formed, where the True value corresponds to the numeric value in the original column. Then the values ​​in the 'no_of_ratings' column are recalculated for values ​​matching the True of the 'correct_no_of_ratings' column

In [13]:
# Add column 'correct_no_of_ratings' which value is 'True' if 'no_of_ratings' begins from digit 
mod_df['no_of_ratings'] = mod_df['no_of_ratings'].astype(str)
mod_df['correct_no_of_ratings'] = pd.Series([mod_df['no_of_ratings'][x][0].isdigit() for x in range(len(mod_df['no_of_ratings']))])
# Drop columns with incorrect 'no_of_ratings'
mod_df = mod_df[mod_df['correct_no_of_ratings'] == True]
mod_df['correct_no_of_ratings'].value_counts()

True    32195
Name: correct_no_of_ratings, dtype: int64

In [14]:
# Change the type to float
mod_df["no_of_ratings"] = mod_df["no_of_ratings"].str.replace(',', '').astype(float)

<ipython-input-14-ef8eca6b37f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_df["no_of_ratings"] = mod_df["no_of_ratings"].str.replace(',', '').astype(float)


In [15]:
# Dataframe after first phase of cleaning
mod_df.head()

,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,correct_no_of_ratings
0,0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255.0,32999.0,58990.0,True
1,1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948.0,46490.0,75990.0,True
2,2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206.0,34490.0,61990.0,True
3,3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69.0,37990.0,68990.0,True
4,4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630.0,34490.0,67790.0,True


In [16]:
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32195 entries, 0 to 37163
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             32195 non-null  int64  
 1   name                   32195 non-null  object 
 2   main_category          32195 non-null  object 
 3   sub_category           32195 non-null  object 
 4   image                  32195 non-null  object 
 5   link                   32195 non-null  object 
 6   ratings                32195 non-null  float64
 7   no_of_ratings          32195 non-null  float64
 8   discount_price         30138 non-null  float64
 9   actual_price           31886 non-null  float64
 10  correct_no_of_ratings  32195 non-null  bool   
dtypes: bool(1), float64(4), int64(1), object(5)
memory usage: 2.7+ MB


In [17]:
# Plot the total missing values
x = mod_df.isnull().sum()

fig = px.bar(x, orientation = "h",  text_auto='.2s',
            color_discrete_sequence= ["#ff6b00"] * len(x))
fig.update_layout(
    title="<b>Missing Value Count</b>",
    xaxis_title="Total missing values",
    yaxis_title="Column Names",
    plot_bgcolor = "#ECECEC",
    showlegend=False
)
fig.show()

# EDA

Now let us imagine we are browsing the Amazon website. What are the things that you see when you click on a product. For me the priority order is as follows:
1. Price
2. Rating
3. Manufacturer
4. Description
5. Customer reviews<br>
Let us see analyze the given dataframe on following points.

In [18]:
# Let us check and create a dataframe of missing ratings
missing_no_of_ratings = mod_df[mod_df['actual_price'].isnull()]

missing_no_of_ratings.head(2)

,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,correct_no_of_ratings
76,76,LG 1.5 Ton 3 Star Hot & Cold DUAL Inverter Spl...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51hbo8yQ1E...,https://www.amazon.in/LG-Inverter-Convertible-...,4.0,265.0,NaN,NaN,True
100,100,Hitachi 1.5 Ton 5 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41AY1pk5oR...,https://www.amazon.in/Hitachi-Inverter-Copper-...,3.8,748.0,NaN,NaN,True


In [19]:
# Since our further analysis is based on the price column so let us drop it.
df = mod_df.dropna(subset=['actual_price','discount_price'])
df.head()

,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,correct_no_of_ratings
0,0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255.0,32999.0,58990.0,True
1,1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948.0,46490.0,75990.0,True
2,2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206.0,34490.0,61990.0,True
3,3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69.0,37990.0,68990.0,True
4,4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630.0,34490.0,67790.0,True


# Extract the manufacturer from the 'name' column
Extract the manufacturer from the 'name' column and insert the 'manufacturer' column after the 'name' column.
To do this, we convert the 'name' column (type 'Series') into a string, split by spaces and select the first substring. We will have some incorrect names (such as 'The', 'Van', etc.) for brands which names consists of more than one word. But for many others it is okay. And I think this is enought to obtain general understanding for our purposes.

In [20]:
df['manufacturer'] = df['name'].str.split(' ').str[0]
cols = df.columns.tolist()
cols

<ipython-input-20-b962e0f71004>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



['Unnamed: 0',
 'name',
 'main_category',
 'sub_category',
 'image',
 'link',
 'ratings',
 'no_of_ratings',
 'discount_price',
 'actual_price',
 'correct_no_of_ratings',
 'manufacturer']

In [21]:
cols = ['name',
 'manufacturer',
 'main_category',
 'sub_category',
 'image',
 'link',
 'ratings',
 'no_of_ratings',
 'discount_price',
 'actual_price']

In [22]:
df = df[cols]
df.head()

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,Lloyd,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255.0,32999.0,58990.0
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,LG,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948.0,46490.0,75990.0
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,LG,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206.0,34490.0,61990.0
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,LG,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69.0,37990.0,68990.0
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,Carrier,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630.0,34490.0,67790.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30138 entries, 0 to 37163
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            30138 non-null  object 
 1   manufacturer    30138 non-null  object 
 2   main_category   30138 non-null  object 
 3   sub_category    30138 non-null  object 
 4   image           30138 non-null  object 
 5   link            30138 non-null  object 
 6   ratings         30138 non-null  float64
 7   no_of_ratings   30138 non-null  float64
 8   discount_price  30138 non-null  float64
 9   actual_price    30138 non-null  float64
dtypes: float64(4), object(6)
memory usage: 2.5+ MB


In [24]:
# Make column with discount net value and discounting percent 
df['discount_value'] = df['actual_price'] - df['discount_price']
df['discounting_percent'] = 1 - df['discount_price']/df['actual_price']

<ipython-input-24-3d1a005e72ba>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
df.head()

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,Lloyd,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255.0,32999.0,58990.0,25991.0,0.440600
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,LG,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948.0,46490.0,75990.0,29500.0,0.388209
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,LG,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206.0,34490.0,61990.0,27500.0,0.443620
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,LG,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69.0,37990.0,68990.0,31000.0,0.449340
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,Carrier,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630.0,34490.0,67790.0,33300.0,0.491223


In [26]:
# Let us check the manufactures according to their prices
df[["actual_price", 'manufacturer']].groupby("manufacturer").mean().round(2).sort_values(by = "actual_price",
                                                                    ascending = False)

,actual_price
manufacturer,
ROCKWELL,110489.0
O,85260.0
MITSUBISHI,83000.0
Vu,80000.0
Proposed_Value,71999.0
...,...
Cavin's,40.0
Lay’s,30.0
Ching's,25.0


In [27]:
# Detail of the maximum price row
df[df["actual_price"] == df["actual_price"].max()]

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent
17595,"Samsung Galaxy S23 Ultra 5G (Green, 12GB, 512G...",Samsung,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/61VfL-aiTo...,https://www.amazon.in/Samsung-Galaxy-Ultra-Gre...,4.4,91.0,134999.0,161999.0,27000.0,0.166668


In [28]:
# Detail of the minimum price row
df[df["discount_value"] == df["discount_value"].min()]

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent
28279,Aristocrat Dual Edge 65cm Polycarbonate & Poly...,Aristocrat,stores,Amazon Fashion,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Aristocrat-Polycarbonate...,4.2,508.0,2823.0,2823.05,0.05,0.000018


In [29]:
# Let us check the common manufacture
values = df["manufacturer"].value_counts().keys().tolist()[:10]
counts = df["manufacturer"].value_counts().tolist()[:10]

In [30]:
fig = px.bar(df, y = counts, x = values,
            color_discrete_sequence = ["#EC2781"] * len(df))


fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  yaxis_title = "Count",
                xaxis_title = "Name of Manufacturers",
                  title = "<b>Popular Manufacturers Category</b>"
                 )
fig.show()

# ✔️ Insight 1
From above graph we see that the Puma is most popular. Let us check the main category for the above top 10 brands

In [31]:
# Creating the dataframe of top 10 manufacturer
df_list = []
for i in values:
    x = df[df["manufacturer"] == i]
    df_list.append(x)
frame = pd.concat(df_list)
frame.head(2)

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent
65,"Amazon Basics 1 Ton, 5 Star, Wi-Fi Enabled Sma...",Amazon,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51s3yrWntM...,https://www.amazon.in/AmazonBasics-1-5-Ton-Air...,3.6,88.0,28990.0,49089.0,20099.0,0.409440
83,"Amazon Basics 1.5 Ton, 5 Star, Wi-Fi Enabled S...",Amazon,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51s3yrWntM...,https://www.amazon.in/AmazonBasics-1-5-Ton-Air...,3.4,39.0,34990.0,56179.0,21189.0,0.377169


In [32]:
# Average rating of the manufactures
frame[["manufacturer", "ratings"]].groupby("manufacturer").mean().sort_values(by = "ratings",
                                                ascending = False)

,ratings
manufacturer,
TheGiftKart,4.323985
Spigen,4.309852
LG,4.192208
Samsung,4.142745
AmazonBasics,4.126705
Havells,4.056545
boAt,4.026360
Amazon,3.983221
Bajaj,3.935393


# ✔️ Insight 2
Even though the most popular brand is Puma but the highest rated is Amazon.<br>
Also the manufacturer Amazon is second most favourite in the popular manufacturer category. On the other hand Puma is second in terms of average rating<br>
Let us now check the popular main category items present.

In [33]:
# Different main categories present
frame["main_category"].unique()

array(['appliances', 'car & motorbike', 'tv, audio & cameras',
       'sports & fitness', 'grocery & gourmet foods', 'home & kitchen',
       'pet supplies', 'stores', "kids' fashion", 'toys & baby products',
       'bags & luggage', 'accessories'], dtype=object)

In [34]:
fig = px.bar(frame, "main_category", 
             color_discrete_sequence = ["#2377a4"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  yaxis_title = "Count",
                  xaxis_title = "Main Categories",
                  title = "<b>Count of Main Categories of Products</b>"
                 )
fig.show()

# ✔️ Insight 3
From the graph we see that 'Man's cloothing' is the popular main category. Let us select the top 10 popular main category. We are narrowing our selction to reach the goal

In [35]:
# Let us select the 5 popular main categories

value_main = frame["main_category"].value_counts().keys().tolist()[:5]
count_main = frame["main_category"].value_counts().tolist()[:5]
value_main

['tv, audio & cameras',
 'appliances',
 'home & kitchen',
 'stores',
 'sports & fitness']

Let us create a new dataframe having top 10 popular manufcturers and 5 most popular main category.

In [36]:
df_list = []
for i in value_main:
    x = frame[frame["main_category"] == i]
    df_list.append(x)
    #print(df)
frame = pd.concat(df_list)
frame.head(2)

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent
11892,Amazon Basics Magic Slate 8.5-inch LCD Writing...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Basics-8-5-inch-W...,4.0,804.0,309.0,999.0,690.0,0.690691
12034,Amazon Basics Portable Multimedia Speaker| Sui...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Basics-Portable-M...,3.7,218.0,399.0,1299.0,900.0,0.692841


In [37]:
# Let us check the popular subcategory
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
frame_sub = frame[["main_category", "sub_category"]].groupby("main_category").count()
frame_sub.style.background_gradient(cmap=cm)


,sub_category
main_category,
appliances,795
home & kitchen,100
sports & fitness,56
stores,59
"tv, audio & cameras",1225


In [38]:
value_sub = frame["sub_category"].value_counts().keys().tolist()[:10]
count_sub = frame["sub_category"].value_counts().tolist()[:10]

In [39]:
# New dataframe with selected sub_category
df_list = []
for i in value_sub:
    x = frame[frame["sub_category"] == i]
    df_list.append(x)
frame = pd.concat(df_list)
frame.head(2)

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent
11892,Amazon Basics Magic Slate 8.5-inch LCD Writing...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Basics-8-5-inch-W...,4.0,804.0,309.0,999.0,690.0,0.690691
12034,Amazon Basics Portable Multimedia Speaker| Sui...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Basics-Portable-M...,3.7,218.0,399.0,1299.0,900.0,0.692841


Now we have completed the second phase of data preprocessing. After this we have achieved a dataframe with following characterstics:
1. No null price.
2. Top 10 manufacturers with respect to count
3. Popular top 5 main categories, 10 sub_category <br>
Let us now check the average rating and price for this selected dataframe.

In [40]:
# Rating of the products
print("The average rating: ",frame["ratings"].unique())

# After processing our data we have significantly reduced the size of the dataframe.
# Also the rating are now 4 or greater.
# Let us now check new average price ### check above before processing to compare.
print("The average price: ", frame["actual_price"].mean())



The average rating:  [4.  3.7 4.4 3.6 3.9 4.5 4.1 4.2 3.8 3.5 4.3 4.6 3.4 3.3 3.1 3.2 4.8 5.
 4.7 4.9 3.  1.  1.4 2.6 2.3 2.8 2.9 2.4 2.7 1.7 2.5]
The average price:  9827.661297539149


In [41]:
import plotly.figure_factory as ff
x = frame["actual_price"]
hist_data = [x]
group_labels = ['actual_price']

fig = ff.create_distplot(hist_data, group_labels, show_rug = False,
                        colors=["#ffd514"])
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  title = "<b>Price Distribution of Data</b>"
                 )

fig.show()

# ✔️ Insight 4
We see that the plot is right skew plot with the presence of outliers. Let us see these outliers

In [42]:
# Check the statistics of the price_new column
frame.actual_price.describe()

count      2235.000000
mean       9827.661298
std       19024.102972
min          69.000000
25%         999.000000
50%        2099.000000
75%        6990.000000
max      161999.000000
Name: actual_price, dtype: float64

In [43]:
# plot the quartiles and check for outliers 
fig = px.box(frame, "actual_price")
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  title = "<b>Price Data Distribution</b>",
                 xaxis_title = "Price of Products"
                 )
fig.show()

In [44]:
# Let us find the outliers
Q1 = 1399
Q2 = 2199
Q3 = 3599
IQR = Q3 - Q1
outlier1 = (Q1 - 1.5 * IQR)
outlier2 = (Q3 + 1.5 * IQR)
print("outlier1: ", outlier1)
print("outlier2: ", outlier2)

outlier1:  -1901.0
outlier2:  6899.0


In [45]:
outlier_price = []

for i in frame.actual_price:
    if i < outlier1 or i > outlier2:
        outlier_price.append("outlier")
    elif i > outlier1 or i < outlier2:
        outlier_price.append("normal")
    
frame["outlier_price"] = outlier_price

In [46]:
fig = px.pie(frame, names = frame["outlier_price"], color = frame["outlier_price"],
             color_discrete_map={'normal': '#2377a4', 'outlier': '#ffd514'})

fig.update_layout(title = "<b>Distribution of Outlier</b>")

fig.show()

In [47]:
# Let us see the outlier value
frame_outlier = frame.loc[frame["outlier_price"] == "outlier"].head()
frame_outlier

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent,outlier_price
20170,Amazon Basics 21.5-Inch (54.5cm) LCD 1920 x 10...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/91Mjsd471N...,https://www.amazon.in/AmazonBasics-21-Inch-Res...,3.9,102.0,5939.0,13500.0,7561.0,0.560074,outlier
11571,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...",Samsung,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81I3w4J6yj...,https://www.amazon.in/Samsung-Mystique-Storage...,4.1,24863.0,15999.0,24999.0,9000.0,0.360014,outlier
11577,"Samsung Galaxy M33 5G (Emerald Brown, 6GB, 128...",Samsung,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81lnKynSaq...,https://www.amazon.in/Samsung-Emerald-Storage-...,4.1,24863.0,15999.0,24999.0,9000.0,0.360014,outlier
11611,"Samsung Galaxy M04 Light Green, 4GB RAM, 64GB ...",Samsung,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/814ePfNubR...,https://www.amazon.in/Samsung-Galaxy-Storage-M...,4.0,1754.0,8499.0,11999.0,3500.0,0.291691,outlier
11636,"Samsung Galaxy M04 Dark Blue, 4GB RAM, 64GB St...",Samsung,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81t6Av5DvX...,https://www.amazon.in/Samsung-Galaxy-Storage-M...,4.0,1754.0,8499.0,11999.0,3500.0,0.291691,outlier


In [48]:
print("Manufacturers with outlier price: ", frame_outlier.manufacturer.value_counts())

Manufacturers with outlier price:  Samsung    4
Amazon     1
Name: manufacturer, dtype: int64


In [49]:
# Top category
print("Main category with outliers: ", frame_outlier.main_category.value_counts())

Main category with outliers:  tv, audio & cameras    5
Name: main_category, dtype: int64


In [50]:
print("Sub_category with outliers: ",frame_outlier.sub_category.value_counts())

Sub_category with outliers:  All Electronics    5
Name: sub_category, dtype: int64


In [51]:
# Let us check the rating of the products
fig = px.violin(frame, "ratings", 
               color_discrete_sequence = ["#FFBF00"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Rating",
                  title = "<b>Rating Distribution of the Popular Products</b>"
                 )
fig.show()

# ✔️ Insight 5
As expected our selected category of products have the most common rating as 4 and 5. It also seems that our selection of top manufacturer, categories have resulted in dataframe having a very few 1, 2 and 3 ratings. Now let us check the customer reviews

In [52]:
fig = px.histogram(frame, "no_of_ratings",
                  color_discrete_sequence = ["#8B4000"] * len(frame))
fig.update_xaxes(range=[10, 5000])
fig.update_yaxes(range=[0, 2000])
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Number of Reviews",
                  title = "<b>Number of Reviews Distribution</b>"
                 )
fig.show()

# ✔️ Insight 6
We see that 10k+ products has less than 49 reviews, 1.5k products has less than 99 reviews. After 1k reviews we see outliers, possibly with fake reviews.

In [53]:
# Let us check if there are any null review
print("Number of null values: ",frame['no_of_ratings'].isnull().sum())
# It seems that with high end products people love to leave a review

Number of null values:  0


In [54]:
frame.head(2)

,name,manufacturer,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_value,discounting_percent,outlier_price
11892,Amazon Basics Magic Slate 8.5-inch LCD Writing...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Basics-8-5-inch-W...,4.0,804.0,309.0,999.0,690.0,0.690691,normal
12034,Amazon Basics Portable Multimedia Speaker| Sui...,Amazon,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Basics-Portable-M...,3.7,218.0,399.0,1299.0,900.0,0.692841,normal


In [55]:
frame['no_of_ratings']

11892       804.0
12034       218.0
12038    437651.0
12173    437651.0
12336     36689.0
           ...   
25788       137.0
25922       250.0
26185     20247.0
35476        36.0
35489        35.0
Name: no_of_ratings, Length: 2235, dtype: float64

In [56]:
fig = px.scatter(frame, x="discounting_percent", y="no_of_ratings", 
                 trendline="ols")
fig.update_yaxes(range=[0, 1000])
fig.update_layout(title = "<b>Relationship between the number of reviews and discount percent</b>",
                 plot_bgcolor = "#ECECEC",
                 yaxis_title = "Number of reviews",
                 xaxis_title = "Discount percent")

fig.show()

# ✔️ Insight 7
We see OLS trendline of slightly decreasing number of reviews with increasing of discount percent. 

In [57]:
fig = px.histogram(frame, "discounting_percent",
                  color_discrete_sequence = ["#C04000"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Discounting Percent",
                  title = "<b>Number of products with different discount percent</b>"
                 )
fig.show()

# 💡 Conclusion
From above analysis of the selected frame of popular categories we arrive at following conclusions:
1. The products with price less than ₹3600 are popular.
2. The outlier in price data are around 5.32%
3. Puma and Amazon are the most popular manufactures with outlier price
4. Character and brand is the subcatogory with outlier price
6. The maximum number rating of popular brands is in range of 4 star
7. Mostly 0-49 review were given on the products
8. Every product has a review in selected dataframe
9. Distribution of products by discount percent shows distribution similar to normal with slight rigth skew and spikes at each value multiple of ten
We can analyse at other relationship as well but for now the above is the conclusion.

